# Library

In [1]:
import tensorflow  as tf
import pandas  as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import random

In [2]:
from textblob import TextBlob
from nltk.tokenize.toktok import ToktokTokenizer
import re
tokenizer=ToktokTokenizer()
import spacy
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer


import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import spacy

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nlp = spacy.load("en_core_web_sm")

nlp=spacy.load('en_core_web_sm',disable=['ner'])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tazkiiia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tazkiiia\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Tazkiiia\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Courses

In [3]:
# read the data
courses= pd.read_csv(r'D:\Recommender_System_Courses_web\dataset\Coursera_courses.csv')
courses.head()

,name,institution,course_url,course_id
0,Machine Learning,Stanford University,https://www.coursera.org/learn/machine-learning,machine-learning
1,Indigenous Canada,University of Alberta,https://www.coursera.org/learn/indigenous-canada,indigenous-canada
2,The Science of Well-Being,Yale University,https://www.coursera.org/learn/the-science-of-...,the-science-of-well-being
3,Technical Support Fundamentals,Google,https://www.coursera.org/learn/technical-suppo...,technical-support-fundamentals
4,Become a CBRS Certified Professional Installer...,Google - Spectrum Sharing,https://www.coursera.org/learn/google-cbrs-cpi...,google-cbrs-cpi-training


# Rating

In [4]:
# read the dataset
data_ratings = pd.read_csv(r'D:\Recommender_System_Courses_web\dataset\Coursera_reviews.csv')
data_ratings.head()

,reviews,reviewers,date_reviews,rating,course_id
0,"Pretty dry, but I was able to pass with just t...",By Robert S,"Feb 12, 2020",4,google-cbrs-cpi-training
1,would be a better experience if the video and ...,By Gabriel E R,"Sep 28, 2020",4,google-cbrs-cpi-training
2,Information was perfect! The program itself wa...,By Jacob D,"Apr 08, 2020",4,google-cbrs-cpi-training
3,A few grammatical mistakes on test made me do ...,By Dale B,"Feb 24, 2020",4,google-cbrs-cpi-training
4,Excellent course and the training provided was...,By Sean G,"Jun 18, 2020",4,google-cbrs-cpi-training


In [5]:
data_ratings.drop(labels = 'date_reviews', inplace=True, axis = 1)
unknown_reviewer_df = data_ratings[~(data_ratings['reviewers'].str.startswith('By'))]
unknown_reviewer_df

,reviews,reviewers,rating,course_id
732410,The ethics unit examples were interesting. Th...,Started Oct 05,4,intro-to-healthcare
732413,Wonderfully done,Started Oct 05,4,intro-to-healthcare
1046787,My certificate does not want to be released,Started Sep 28,1,entrepreneurship-strategy
1046790,knowing who you are? give me a break...,Started Sep 28,1,entrepreneurship-strategy
1236372,"boring lecturer, monotone, passionless reading...",No Upcoming Session Available,1,epigenetics
1236375,the whole lecture given by the lecturer is so ...,No Upcoming Session Available,1,epigenetics
1259484,hola! me gustaría darme de baja de este curso....,Started Oct 05,1,social-economic-networks
1259487,"Without previous knowledge in math, this cours...",Started Oct 05,1,social-economic-networks
1262903,How can you attend yo course which does not up...,Started Oct 05,1,curanderismo-plants
1262906,"The course is not what I expected, and I would...",Started Oct 05,1,curanderismo-plants


In [6]:
num_of_reviews_by_course_df = data_ratings['course_id'].value_counts().reset_index()
more_500_reviews_df = num_of_reviews_by_course_df[num_of_reviews_by_course_df['count'] > 500]
more_500_reviews_df

,course_id,count
0,python,45218
1,machine-learning,35895
2,neural-networks-deep-learning,35750
3,technical-support-fundamentals,35319
4,python-data,33546
...,...,...
413,getting-started-with-google-sheets,517
414,quality-healthcare,516
415,process-mining,514
416,international-taxation,513


In [7]:
# Ambil kursus yang hanya memiliki lebih dari 500 reviews
new_reviews_df = data_ratings[data_ratings['course_id']\
                                .isin(set(more_500_reviews_df['course_id']))]

# Ambil data yang reviewernya diawali dengan 'By' kecuali 'By Deleted A'
data_ratings = data_ratings[(data_ratings['reviewers'].str.contains('By')) &
                                (data_ratings['reviewers']!='By Deleted A')]

# Hapus data yang memiliki nilai null
data_ratings.dropna(inplace=True)

# Hapus data duplikat
data_ratings.drop_duplicates(inplace=True)

data_ratings.head()

,reviews,reviewers,rating,course_id
0,"Pretty dry, but I was able to pass with just t...",By Robert S,4,google-cbrs-cpi-training
1,would be a better experience if the video and ...,By Gabriel E R,4,google-cbrs-cpi-training
2,Information was perfect! The program itself wa...,By Jacob D,4,google-cbrs-cpi-training
3,A few grammatical mistakes on test made me do ...,By Dale B,4,google-cbrs-cpi-training
4,Excellent course and the training provided was...,By Sean G,4,google-cbrs-cpi-training


In [8]:
data_ratings['reviewers'] = data_ratings['reviewers']\
                      .apply(lambda x: ' '.join(x.split(' ')[1:]))
data_ratings.head()

,reviews,reviewers,rating,course_id
0,"Pretty dry, but I was able to pass with just t...",Robert S,4,google-cbrs-cpi-training
1,would be a better experience if the video and ...,Gabriel E R,4,google-cbrs-cpi-training
2,Information was perfect! The program itself wa...,Jacob D,4,google-cbrs-cpi-training
3,A few grammatical mistakes on test made me do ...,Dale B,4,google-cbrs-cpi-training
4,Excellent course and the training provided was...,Sean G,4,google-cbrs-cpi-training


In [9]:
merged_df = pd.merge(courses[['name','course_url','course_id']],
                     data_ratings,
                     on='course_id')

merged_df.head()

,name,course_url,course_id,reviews,reviewers,rating
0,Machine Learning,https://www.coursera.org/learn/machine-learning,machine-learning,The course is ok but the certification procedu...,Bruno C,1
1,Machine Learning,https://www.coursera.org/learn/machine-learning,machine-learning,"I just started week 3 , I have to admit that I...",Fadi,1
2,Machine Learning,https://www.coursera.org/learn/machine-learning,machine-learning,This course is absolute garbage. You get no f...,Mathew L,1
3,Machine Learning,https://www.coursera.org/learn/machine-learning,machine-learning,"However good the material and lectures may be,...",Rui C,1
4,Machine Learning,https://www.coursera.org/learn/machine-learning,machine-learning,Too many trainings and assignments without eno...,Mehdi A,1


# Merge Data

In [10]:
data_prep_df = merged_df.copy()

users_list = data_prep_df['reviewers'].unique().tolist()
courses_list = data_prep_df['course_id'].unique().tolist()

# Encode users dan courses
users_encoded = {user: i+1 for i, user in enumerate(users_list)}
courses_encoded = {course: i+1 for i, course in enumerate(courses_list)}

# Decode users dan courses
users_decoded = {i+1: user for i, user in enumerate(users_list)}
courses_decoded = {i+1: course for i, course in enumerate(courses_list)}

In [11]:
data_prep_df['user_id'] = data_prep_df['reviewers'].map(users_encoded)
data_prep_df['course_id'] = data_prep_df['course_id'].map(courses_encoded)

data_prep_df.head()

,name,course_url,course_id,reviews,reviewers,rating,user_id
0,Machine Learning,https://www.coursera.org/learn/machine-learning,1,The course is ok but the certification procedu...,Bruno C,1,1
1,Machine Learning,https://www.coursera.org/learn/machine-learning,1,"I just started week 3 , I have to admit that I...",Fadi,1,2
2,Machine Learning,https://www.coursera.org/learn/machine-learning,1,This course is absolute garbage. You get no f...,Mathew L,1,3
3,Machine Learning,https://www.coursera.org/learn/machine-learning,1,"However good the material and lectures may be,...",Rui C,1,4
4,Machine Learning,https://www.coursera.org/learn/machine-learning,1,Too many trainings and assignments without eno...,Mehdi A,1,5


In [ ]:
#  save the final file
# Save the DataFrame as a CSV file to the specified folder
# data_prep_df.to_csv(r'D:\Recommender_System_Courses_web\dataset\final_data.csv', index=False)


# Preprocessing

In [13]:
# read the data
df = pd.read_csv(r'D:\Recommender_System_Courses_web\dataset\final_data.csv')
df.head(5)

,name,course_url,course_id,reviews,reviewers,rating,user_id
0,Machine Learning,https://www.coursera.org/learn/machine-learning,1,The course is ok but the certification procedu...,Bruno C,1,1
1,Machine Learning,https://www.coursera.org/learn/machine-learning,1,"I just started week 3 , I have to admit that I...",Fadi,1,2
2,Machine Learning,https://www.coursera.org/learn/machine-learning,1,This course is absolute garbage. You get no f...,Mathew L,1,3
3,Machine Learning,https://www.coursera.org/learn/machine-learning,1,"However good the material and lectures may be,...",Rui C,1,4
4,Machine Learning,https://www.coursera.org/learn/machine-learning,1,Too many trainings and assignments without eno...,Mehdi A,1,5


In [14]:
df.dtypes

name          object
course_url    object
course_id      int64
reviews       object
reviewers     object
rating         int64
user_id        int64
dtype: object

In [15]:
df.isnull().sum()

name          0
course_url    0
course_id     0
reviews       0
reviewers     3
rating        0
user_id       0
dtype: int64

In [ ]:
def clean_text(series):
    series = series.apply(lambda x:  re.sub(r'_+', ' ', x))
    series = series.apply(lambda x: re.sub(r'[^a-zA-Z\s]', '', x))
    series = series.str.lower()
    series = series.apply(lambda x: ''.join([c for c in x if c not in punctuation]))
    series = series.apply(lambda x: re.sub(r'\d+', '', x))
    series = series.str.strip()
    series = series.apply(lambda x: re.sub(' +', ' ', x))

    return series
df['reviews']=clean_text(df['reviews'])
df.head()

,name,course_url,course_id,reviews,reviewers,rating,user_id
0,Machine Learning,https://www.coursera.org/learn/machine-learning,1,the course is ok but the certification procedu...,Bruno C,1,1
1,Machine Learning,https://www.coursera.org/learn/machine-learning,1,i just started week i have to admit that it is...,Fadi,1,2
2,Machine Learning,https://www.coursera.org/learn/machine-learning,1,this course is absolute garbage you get no fee...,Mathew L,1,3
3,Machine Learning,https://www.coursera.org/learn/machine-learning,1,however good the material and lectures may be ...,Rui C,1,4
4,Machine Learning,https://www.coursera.org/learn/machine-learning,1,too many trainings and assignments without eno...,Mehdi A,1,5


In [ ]:
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word.lower() not in stop_words]
    return ' '.join(filtered_text)

df['reviews'] = df['reviews'].apply(remove_stopwords)
df.head()

,name,course_url,course_id,reviews,reviewers,rating,user_id
0,Machine Learning,https://www.coursera.org/learn/machine-learning,1,course ok certification procedure messno state...,Bruno C,1,1
1,Machine Learning,https://www.coursera.org/learn/machine-learning,1,started week admit good course explaining idea...,Fadi,1,2
2,Machine Learning,https://www.coursera.org/learn/machine-learning,1,course absolute garbage get feedback quizzes a...,Mathew L,1,3
3,Machine Learning,https://www.coursera.org/learn/machine-learning,1,however good material lectures may use outdate...,Rui C,1,4
4,Machine Learning,https://www.coursera.org/learn/machine-learning,1,many trainings assignments without enough prac...,Mehdi A,1,5


In [ ]:
def lemmatize_text(text):
    text=nlp(text)
    text= ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text
df['reviews'] = df['reviews'].apply(lemmatize_text)
df.head()

,name,course_url,course_id,reviews,reviewers,rating,user_id
0,Machine Learning,https://www.coursera.org/learn/machine-learning,1,course ok certification procedure messno state...,Bruno C,1,1
1,Machine Learning,https://www.coursera.org/learn/machine-learning,1,start week admit good course explain idea hypn...,Fadi,1,2
2,Machine Learning,https://www.coursera.org/learn/machine-learning,1,course absolute garbage get feedback quiz assi...,Mathew L,1,3
3,Machine Learning,https://www.coursera.org/learn/machine-learning,1,however good material lecture may use outdated...,Rui C,1,4
4,Machine Learning,https://www.coursera.org/learn/machine-learning,1,many training assignment without enough practi...,Mehdi A,1,5


In [ ]:
df.to_csv('cleaned_reviews.csv', index=False)

# Sentiment using VADER

In [16]:
df = pd.read_csv(r'D:\Recommender_System_Courses_web\dataset\cleaned_reviews.csv')
df.head(5)

,name,course_url,course_id,reviews,reviewers,rating,user_id
0,Machine Learning,https://www.coursera.org/learn/machine-learning,1,course ok certification procedure messno state...,Bruno C,1,1
1,Machine Learning,https://www.coursera.org/learn/machine-learning,1,start week admit good course explain idea hypn...,Fadi,1,2
2,Machine Learning,https://www.coursera.org/learn/machine-learning,1,course absolute garbage get feedback quiz assi...,Mathew L,1,3
3,Machine Learning,https://www.coursera.org/learn/machine-learning,1,however good material lecture may use outdated...,Rui C,1,4
4,Machine Learning,https://www.coursera.org/learn/machine-learning,1,many training assignment without enough practi...,Mehdi A,1,5


In [ ]:
# pip install vaderSentiment


[notice] A new release of pip is available: 24.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

def get_compound_score(text):
    if isinstance(text, float):  
        text = str(text)  
    vs = analyzer.polarity_scores(text)
    return vs['compound']

df['reviews'] = df['reviews'].fillna('')
df['compound'] = df['reviews'].apply(get_compound_score)

# Function to classify sentiment into Positive, Neutral, Negative
def classify_sentiment(compound):
    if compound > 0.001:
        return 'Positive', 1
    elif -0.001 < compound < 0.001:
        return 'Neutral', 0
    else:
        return 'Negative', -1

# Apply the overall sentiment classification
df[['Sentiment', 'polarity']] = df['compound'].apply(lambda x: pd.Series(classify_sentiment(x)))

# Function to classify sentiment into an integer value from 1 to 5
def refined_sentiment_classification(vader_compound_score):
    if vader_compound_score >= -1 and vader_compound_score < -0.5:
        return 1
    elif vader_compound_score >= -0.5 and vader_compound_score < -0.05:
        return 2
    elif vader_compound_score >= -0.05 and vader_compound_score <= 0.05:
        return 3
    elif vader_compound_score > 0.05 and vader_compound_score < 0.5:
        return 4
    else:
        return 5

# Apply the refined sentiment classification
df['refined_sentiment'] = df['compound'].apply(refined_sentiment_classification)




In [19]:
df.head(5)

,name,course_url,course_id,reviews,reviewers,rating,user_id,compound,Sentiment,polarity,refined_sentiment
0,Machine Learning,https://www.coursera.org/learn/machine-learning,1,course ok certification procedure messno state...,Bruno C,1,1,0.3612,Positive,1,4
1,Machine Learning,https://www.coursera.org/learn/machine-learning,1,start week admit good course explain idea hypn...,Fadi,1,2,0.8360,Positive,1,5
2,Machine Learning,https://www.coursera.org/learn/machine-learning,1,course absolute garbage get feedback quiz assi...,Mathew L,1,3,-0.8555,Negative,-1,1
3,Machine Learning,https://www.coursera.org/learn/machine-learning,1,however good material lecture may use outdated...,Rui C,1,4,0.4404,Positive,1,4
4,Machine Learning,https://www.coursera.org/learn/machine-learning,1,many training assignment without enough practi...,Mehdi A,1,5,-0.2263,Negative,-1,2


In [20]:
df = df.drop(columns=['compound', 'polarity'])
df.head()

,name,course_url,course_id,reviews,reviewers,rating,user_id,Sentiment,refined_sentiment
0,Machine Learning,https://www.coursera.org/learn/machine-learning,1,course ok certification procedure messno state...,Bruno C,1,1,Positive,4
1,Machine Learning,https://www.coursera.org/learn/machine-learning,1,start week admit good course explain idea hypn...,Fadi,1,2,Positive,5
2,Machine Learning,https://www.coursera.org/learn/machine-learning,1,course absolute garbage get feedback quiz assi...,Mathew L,1,3,Negative,1
3,Machine Learning,https://www.coursera.org/learn/machine-learning,1,however good material lecture may use outdated...,Rui C,1,4,Positive,4
4,Machine Learning,https://www.coursera.org/learn/machine-learning,1,many training assignment without enough practi...,Mehdi A,1,5,Negative,2


In [21]:
df = df.sample(frac=1, random_state=42)
df.head()

,name,course_url,course_id,reviews,reviewers,rating,user_id,Sentiment,refined_sentiment
243441,Introductory Human Physiology,https://www.coursera.org/learn/physiology,82,really perfect course really enjoy thank much,Rishikesh D,5,156321,Positive,5
505021,Machine Learning Foundations: A Case Study App...,https://www.coursera.org/learn/ml-foundations,554,great,Md. R Q S,5,19402,Positive,5
467766,Object-Oriented Design,https://www.coursera.org/learn/object-oriented...,437,interesting course well organize,ALESSANDRO S,5,244177,Positive,5
93921,Successful Negotiation: Essential Strategies a...,https://www.coursera.org/learn/negotiation-skills,15,first big thank coursera give opportunity mill...,NOORUDDIN T,5,71606,Positive,5
431323,"Capstone: Retrieving, Processing, and Visualiz...",https://www.coursera.org/learn/python-data-vis...,361,assignment easy expect least code change,Remo L,4,131006,Positive,4


In [22]:
df.to_csv('data_fixed.csv', index=False)


# Modeling

In [4]:
df = pd.read_excel(r'D:\Bismillah SKRIPSIIIII\REVISI\dataset\Dataset_revisi.xlsx')
df.head(5)

,name,course_url,course_id,reviews,reviewers,rating,user_id,Sentiment,refined_sentiment,Sequence,Unnamed: 10,Unnamed: 11
0,Machine Learning Foundations: A Case Study App...,https://www.coursera.org/learn/ml-foundations,554,great,Md. R Q S,5,19402,Positive,5,machine learning foundation case study approac...,NaN,NaN
1,Capstone: Retrieving,Processing,and Visualizing Data with Python,https://www.coursera.org/learn/python-data-vis...,361,assignment easy expect least code change,Remo L,4,131006,Positive,4,capstone retrieving processing visualizing dat...
2,Improving Deep Neural Networks: Hyperparameter...,Regularization and Optimization,https://www.coursera.org/learn/deep-neural-net...,34,great experience learn many insightful thing n...,Rooholla K,5,64547,Positive,5,improving deep neural network hyperparameter t...,NaN
3,Excel Skills for Business: Advanced,https://www.coursera.org/learn/excel-advanced,479,hello allfirst thank much instructor nicky pra...,Neeraj K J,5,85290,Positive,5,excel skill business advanced hello allfirst t...,NaN,NaN
4,Convolutional Neural Networks in TensorFlow,https://www.coursera.org/learn/convolutional-n...,77,course build previous introductory course spec...,Deepak V,5,7713,Positive,5,convolutional neural network tensorflow course...,NaN,NaN


In [5]:
df = df.drop(columns=['Sentiment','refined_sentiment','Unnamed: 10', 'Unnamed: 11'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249082 entries, 0 to 249081
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   name        249082 non-null  object
 1   course_url  249082 non-null  object
 2   course_id   249082 non-null  object
 3   reviews     249082 non-null  object
 4   reviewers   249077 non-null  object
 5   rating      249077 non-null  object
 6   user_id     249081 non-null  object
 7   Sequence    249082 non-null  object
dtypes: object(8)
memory usage: 15.2+ MB


In [6]:
num_of_reviews_by_course_df = df['course_id'].value_counts().reset_index()
more_500_reviews_df = num_of_reviews_by_course_df[num_of_reviews_by_course_df['count'] > 500]
more_500_reviews_df

,course_id,count
0,8,15079
1,1,12591
2,13,12227
3,4,11945
4,28,11323
...,...,...
110,574,538
111,and Dictionaries,531
112,595,521
113,534,509


In [7]:
df = df[df['course_id']\
    .isin(set(more_500_reviews_df['course_id']))]
# Hapus data yang memiliki nilai null
df.dropna(inplace=True)

# Hapus data duplikat
df.drop_duplicates(inplace=True)

df.head()

,name,course_url,course_id,reviews,reviewers,rating,user_id,Sequence
0,Machine Learning Foundations: A Case Study App...,https://www.coursera.org/learn/ml-foundations,554,great,Md. R Q S,5,19402,machine learning foundation case study approac...
1,Capstone: Retrieving,Processing,and Visualizing Data with Python,https://www.coursera.org/learn/python-data-vis...,361,assignment easy expect least code change,Remo L,Positive
2,Improving Deep Neural Networks: Hyperparameter...,Regularization and Optimization,https://www.coursera.org/learn/deep-neural-net...,34,great experience learn many insightful thing n...,Rooholla K,5,5
3,Excel Skills for Business: Advanced,https://www.coursera.org/learn/excel-advanced,479,hello allfirst thank much instructor nicky pra...,Neeraj K J,5,85290,excel skill business advanced hello allfirst t...
4,Convolutional Neural Networks in TensorFlow,https://www.coursera.org/learn/convolutional-n...,77,course build previous introductory course spec...,Deepak V,5,7713,convolutional neural network tensorflow course...


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 249071 entries, 0 to 249081
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   name        249071 non-null  object
 1   course_url  249071 non-null  object
 2   course_id   249071 non-null  object
 3   reviews     249071 non-null  object
 4   reviewers   249071 non-null  object
 5   rating      249071 non-null  object
 6   user_id     249071 non-null  object
 7   Sequence    249071 non-null  object
dtypes: object(8)
memory usage: 17.1+ MB


In [9]:
df['course_id'].nunique()

115

In [10]:
# Menghapus data abnormal
def remove_abnormal_data(df):
    df['rating'] = pd.to_numeric(df['rating'], errors='coerce')
    
    df = df[(df['rating'] >= 1) & (df['rating'] <= 5)]
    
    df = df.dropna(subset=['name', 'course_url', 'reviews', 'user_id', 'rating'])

    if 'refined_sentiment' in df.columns:
        df['refined_sentiment'] = pd.to_numeric(df['refined_sentiment'], errors='coerce')
        df = df[(df['refined_sentiment'] >= 1) & (df['refined_sentiment'] <= 5)]
    
    return df

df_cleaned = remove_abnormal_data(df)
df_cleaned.head()

,name,course_url,course_id,reviews,reviewers,rating,user_id,Sequence
0,Machine Learning Foundations: A Case Study App...,https://www.coursera.org/learn/ml-foundations,554,great,Md. R Q S,5.0,19402,machine learning foundation case study approac...
3,Excel Skills for Business: Advanced,https://www.coursera.org/learn/excel-advanced,479,hello allfirst thank much instructor nicky pra...,Neeraj K J,5.0,85290,excel skill business advanced hello allfirst t...
4,Convolutional Neural Networks in TensorFlow,https://www.coursera.org/learn/convolutional-n...,77,course build previous introductory course spec...,Deepak V,5.0,7713,convolutional neural network tensorflow course...
5,Programming for Everybody (Getting Started wit...,https://www.coursera.org/learn/python,8,want fast ramp programming general python part...,Arthur M M,5.0,39897,programming everybody getting started python w...
6,Data Analytics for Lean Six Sigma,https://www.coursera.org/learn/data-analytics-...,364,way dr inez teach brilliant love example fill ...,Siddarth S,5.0,150081,data analytics lean six sigma way dr inez teac...


In [11]:
df_cleaned = df_cleaned.copy()

users_list = df_cleaned['reviewers'].unique().tolist()
courses_list = df_cleaned['course_id'].unique().tolist()

# Encode users dan courses
users_encoded = {user: i+1 for i, user in enumerate(users_list)}
courses_encoded = {course: i+1 for i, course in enumerate(courses_list)}

# Decode users dan courses
users_decoded = {i+1: user for i, user in enumerate(users_list)}
courses_decoded = {i+1: course for i, course in enumerate(courses_list)}

In [12]:
df_cleaned = df_cleaned.copy()
df_cleaned['user_id'] = df_cleaned['reviewers'].map(users_encoded)
df_cleaned['course_id'] = df_cleaned['course_id'].map(courses_encoded)

df_cleaned.head()

,name,course_url,course_id,reviews,reviewers,rating,user_id,Sequence
0,Machine Learning Foundations: A Case Study App...,https://www.coursera.org/learn/ml-foundations,1,great,Md. R Q S,5.0,1,machine learning foundation case study approac...
3,Excel Skills for Business: Advanced,https://www.coursera.org/learn/excel-advanced,2,hello allfirst thank much instructor nicky pra...,Neeraj K J,5.0,2,excel skill business advanced hello allfirst t...
4,Convolutional Neural Networks in TensorFlow,https://www.coursera.org/learn/convolutional-n...,3,course build previous introductory course spec...,Deepak V,5.0,3,convolutional neural network tensorflow course...
5,Programming for Everybody (Getting Started wit...,https://www.coursera.org/learn/python,4,want fast ramp programming general python part...,Arthur M M,5.0,4,programming everybody getting started python w...
6,Data Analytics for Lean Six Sigma,https://www.coursera.org/learn/data-analytics-...,5,way dr inez teach brilliant love example fill ...,Siddarth S,5.0,5,data analytics lean six sigma way dr inez teac...


In [13]:
def is_valid_rating(value):
    try:
        value = float(value)
        return 1 <= value <= 5
    except ValueError:
        return False

df_cleaned = df_cleaned[df_cleaned['rating'].apply(is_valid_rating)]

df_cleaned['rating'] = df_cleaned['rating'].astype(np.float32)

# Menentukan jumlah pengguna, jumlah kursus, min, dan max rating
users_encoded = df_cleaned['user_id'].unique()
courses_encoded = df_cleaned['course_id'].unique()

num_users = len(users_encoded)
num_courses = len(courses_encoded)

min_rating = min(df_cleaned['rating'])
max_rating = max(df_cleaned['rating'])

print(f'Number of User: {num_users}\nNumber of Course: {num_courses}\nMin Rating: {min_rating}\nMax Rating: {max_rating}')


Number of User: 137187
Number of Course: 109
Min Rating: 1.0
Max Rating: 5.0


In [14]:
df_cleaned.to_csv('data_cleaned.csv', index=False)
df_cleaned.head()

,name,course_url,course_id,reviews,reviewers,rating,user_id,Sequence
0,Machine Learning Foundations: A Case Study App...,https://www.coursera.org/learn/ml-foundations,1,great,Md. R Q S,5.0,1,machine learning foundation case study approac...
3,Excel Skills for Business: Advanced,https://www.coursera.org/learn/excel-advanced,2,hello allfirst thank much instructor nicky pra...,Neeraj K J,5.0,2,excel skill business advanced hello allfirst t...
4,Convolutional Neural Networks in TensorFlow,https://www.coursera.org/learn/convolutional-n...,3,course build previous introductory course spec...,Deepak V,5.0,3,convolutional neural network tensorflow course...
5,Programming for Everybody (Getting Started wit...,https://www.coursera.org/learn/python,4,want fast ramp programming general python part...,Arthur M M,5.0,4,programming everybody getting started python w...
6,Data Analytics for Lean Six Sigma,https://www.coursera.org/learn/data-analytics-...,5,way dr inez teach brilliant love example fill ...,Siddarth S,5.0,5,data analytics lean six sigma way dr inez teac...


Reduce Data 

In [2]:
df = pd.read_csv(r'D:\Bismillah SKRIPSIIIII\REVISI\dataset\data_cleaned.csv')
df.head()

,name,course_url,course_id,reviews,reviewers,rating,user_id,Sequence
0,Machine Learning Foundations: A Case Study App...,https://www.coursera.org/learn/ml-foundations,1,great,Md. R Q S,5.0,1,machine learning foundation case study approac...
1,Excel Skills for Business: Advanced,https://www.coursera.org/learn/excel-advanced,2,hello allfirst thank much instructor nicky pra...,Neeraj K J,5.0,2,excel skill business advanced hello allfirst t...
2,Convolutional Neural Networks in TensorFlow,https://www.coursera.org/learn/convolutional-n...,3,course build previous introductory course spec...,Deepak V,5.0,3,convolutional neural network tensorflow course...
3,Programming for Everybody (Getting Started wit...,https://www.coursera.org/learn/python,4,want fast ramp programming general python part...,Arthur M M,5.0,4,programming everybody getting started python w...
4,Data Analytics for Lean Six Sigma,https://www.coursera.org/learn/data-analytics-...,5,way dr inez teach brilliant love example fill ...,Siddarth S,5.0,5,data analytics lean six sigma way dr inez teac...


In [4]:
# Hitung jumlah unik course_id
unique_courses = df['course_id'].nunique()

# Hitung jumlah data per course_id
samples_per_course = 2000 // unique_courses

# Sampling data secara stratified
df_sampled = df.groupby('course_id').apply(lambda x: x.sample(min(len(x), samples_per_course))).reset_index(drop=True)

# Jika jumlah total belum mencapai 2000, ambil tambahan data secara random
if len(df_sampled) < 2000:
    remaining_samples = 2000 - len(df_sampled)
    additional_samples = df[~df.index.isin(df_sampled.index)].sample(n=remaining_samples)
    df_sampled = pd.concat([df_sampled, additional_samples])

# Hasil akhir
print(df_sampled.shape)

(2000, 8)


C:\Users\Tazkiiia\AppData\Local\Temp\ipykernel_12516\2834703549.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sampled = df.groupby('course_id').apply(lambda x: x.sample(min(len(x), samples_per_course))).reset_index(drop=True)


In [5]:
# Menyimpan data sampled ke dalam file CSV
output_path = r'D:\Bismillah SKRIPSIIIII\REVISI\dataset\data_sampled.csv'
df_sampled.to_csv(output_path, index=False)

print(f"Data berhasil disimpan ke {output_path}")


Data berhasil disimpan ke D:\Bismillah SKRIPSIIIII\REVISI\dataset\data_sampled.csv
